In [ ]:
# Colab code
# Mount data
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Colab code 
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

Thu Aug  8 00:45:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0              55W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# Colab code: Install packages
# !pip install umap-learn bertopic hdbscan scikit-learn

In [1]:
# import packages
import pandas as pd
import numpy as np
import json

# UMAP library for dimensionality reduction
from umap import UMAP

# Plotting library
import matplotlib.pyplot as plt


# Topic modelling
from bertopic import BERTopic
from hdbscan import HDBSCAN

# Removing stop words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\hoath\Git\LobbyMap_ML\env_python10\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\hoath\Git\LobbyMap_ML\env_python10\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\hoath\Git\LobbyMap_ML\env_python10\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\hoath\Git\LobbyMap_ML\env_python10\lib\site-packages\traitlets\config\application.py", line 1075, in l

In [2]:
from importlib.metadata import version
version('bertopic')

'0.16.3'

In [3]:
# Load Json data:

# file_path = '/content/drive/MyDrive/combined.jsonl' # in Colab

import os
file_path = r'C:\Users\hoath\Git\LobbyMap_ML\data\processed\combined.jsonl'


# Load JSONL data
data = []
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

df = pd.DataFrame(data)

print(df.iloc[0])


document_id    -000-985-AES_nytimes_CEOnuclear_15.6.22_june20...
sentences      [{'sentence_id': 0, 'page_idx': 0, 'block_idx'...
evidences      [{'query': 'alignment_with_ipcc_on_climate_act...
meta           {'parser': 'fitz', 'evidences': [[{'company_na...
Name: 0, dtype: object


In [5]:
# Function to extract 'evidence_year' and 'evidence_pdf_urls' from nested 'meta' -> 'evidences'
def extract_evidence_info(entry):
    evidence_info = []
    if 'meta' in entry and 'evidences' in entry['meta']:
        for evidence_group in entry['meta']['evidences']:
            for evidence in evidence_group:  # Assuming each evidence_group is a list of evidences
                year = evidence.get('evidence_year', None)
                urls = evidence.get('evidence_pdf_urls', None)
                if year == 105:
                    evidence_info.append((year, urls))
    return evidence_info

# Apply the function to extract the required info
df['evidence_info'] = df.apply(extract_evidence_info, axis=1)

# Now filter the DataFrame for entries where 'evidence_year' contains 105
filtered_df = df[df['evidence_info'].apply(lambda info: any(year == 105 for year, urls in info))]

# Print the filtered information: evidence_year and evidence_pdf_urls
for index, row in filtered_df.iterrows():
    for year, urls in row['evidence_info']:
        if year == 105:
            print(f"Evidence Year: {year}")
            print(f"Evidence PDF URLs: {urls}")
            print("-" * 50)  # Separator between entries

Evidence Year: 105
Evidence PDF URLs: ['https://lobbymap.org/site//data/000/072/UnitedTechExtract_CDP2014_July_24_2015.pdf']
--------------------------------------------------


In [6]:
# This snipet is to check the url of document by year to see if there is any issues with the 
# the data entry of the evidence_year

import json
import pandas as pd
from collections import defaultdict

def count_unique_ids_and_words_by_year(input_file):
    year_data = defaultdict(lambda: {'unique_documents': set(), 'unique_sentences': set(), 'word_count': 0})
    
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            data = json.loads(line)
            document_id = data.get('document_id')
            sentences = data.get('sentences', [])
            meta = data.get('meta', {})
            meta_evidences = meta.get('evidences', [])
            
            # Extract evidence years and update sets
            for evidence_list in meta_evidences:
                for evidence in evidence_list:
                    evidence_year = evidence.get('evidence_year')

                     # Replace evidence_year with 2015 if it is 105
                    if evidence_year == 105:
                        evidence_year = 2015
                        
                    if evidence_year:
                        year_data[evidence_year]['unique_documents'].add(document_id)
                        for sentence in sentences:
                            year_data[evidence_year]['unique_sentences'].add((document_id, sentence['sentence_id']))
                            year_data[evidence_year]['word_count'] += len(sentence['text'].split())
                            
    return year_data

def extract_pdf_urls_by_year(input_file, target_years):
    pdf_urls_by_year = defaultdict(list)
    
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            data = json.loads(line)
            meta = data.get('meta', {})
            meta_evidences = meta.get('evidences', [])
            
            # Extract evidence_pdf_urls for the target years
            for evidence_list in meta_evidences:
                for evidence in evidence_list:
                    evidence_year = evidence.get('evidence_year')
                    
                    # Replace evidence_year with 2015 if it is 105
                    if evidence_year == 105:
                        evidence_year = 2015
                    
                    if evidence_year in target_years:
                        pdf_urls = evidence.get('evidence_pdf_urls', [])
                        if pdf_urls:
                            pdf_urls_by_year[evidence_year].extend(pdf_urls)
                            
    return pdf_urls_by_year

# Define the path to your combined file
combined_file = r'C:\Users\hoath\Git\LobbyMap_ML\data\processed\combined.jsonl'

# Count unique document_id, sentence_id, and words by year
year_data = count_unique_ids_and_words_by_year(combined_file)

# Extract PDF URLs for the years 2005 and 2009
pdf_urls_by_year = extract_pdf_urls_by_year(combined_file, target_years=[2005, 2009])

# Print the PDF URLs for the years 2005 and 2009
for year, urls in pdf_urls_by_year.items():
    print(f"PDF URLs for Year {year}:")
    for url in urls:
        print(url)
    print("-" * 50)


PDF URLs for Year 2009:
https://lobbymap.org/site//data/000/733/05032010_lafarge_sustainability_report_2009-fr.pdf
https://lobbymap.org/site//data/000/075/101_Duke-Energy---Q2-:-D5_75830_2015-07-29_22:25.pdf
--------------------------------------------------
PDF URLs for Year 2005:
https://lobbymap.org/site//data/000/900/Daikin_Sustainability_investor_presentation_script_Dec_2021.pdf
--------------------------------------------------


In [7]:
# get a count table of docs, sent and words by evidence_year
import json
import pandas as pd
from collections import defaultdict

def count_unique_ids_and_words_by_year(input_file):
    year_data = defaultdict(lambda: {'unique_documents': set(), 'unique_sentences': set(), 'word_count': 0})
    
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            data = json.loads(line)
            document_id = data.get('document_id')
            sentences = data.get('sentences', [])
            meta = data.get('meta', {})
            meta_evidences = meta.get('evidences', [])
            
            # Extract evidence years and update sets
            for evidence_list in meta_evidences:
                for evidence in evidence_list:
                    evidence_year = evidence.get('evidence_year')

                     # Replace evidence_year with 2015 if it is 105
                    if evidence_year == 105:
                        evidence_year = 2015
                        
                    if evidence_year:
                        year_data[evidence_year]['unique_documents'].add(document_id)
                        for sentence in sentences:
                            year_data[evidence_year]['unique_sentences'].add((document_id, sentence['sentence_id']))
                            year_data[evidence_year]['word_count'] += len(sentence['text'].split())

    # Convert sets to counts
    year_counts = {year: {'unique_documents': len(data['unique_documents']),
                          'unique_sentences': len(data['unique_sentences']),
                          'word_count': data['word_count']}
                   for year, data in year_data.items()}
    
    return year_counts

# Define the path to your combined file
combined_file = file_path

# Count unique document_id, sentence_id, and words by year
year_counts = count_unique_ids_and_words_by_year(combined_file)

# Create a DataFrame from the counts
df = pd.DataFrame.from_dict(year_counts, orient='index')
df.index.name = 'Year'
df.columns = ['Number of Unique Document IDs', 'Number of Unique Sentence IDs', 'Total Word Count']

# # Define the path to save the CSV file
# csv_file_path = 'data/processed/yearly_counts.csv'

# # Save the DataFrame to a CSV file
# df.to_csv(csv_file_path)

# # Display the DataFrame
# print(df)

# Generate LaTeX source code
latex_code = df.to_latex(index=True, caption="Yearly Counts of Documents, Sentences, and Words")

# Define the path to save the LaTeX source code
latex_file_path = r'C:\Users\hoath\Git\LobbyMap_ML\data\processed\yearly_counts.tex'

# Save the LaTeX source code to a file
with open(latex_file_path, 'w') as latex_file:
    latex_file.write(latex_code)

# Print the LaTeX code to the console
print(latex_code)



\begin{table}
\caption{Yearly Counts of Documents, Sentences, and Words}
\begin{tabular}{lrrr}
\toprule
 & Number of Unique Document IDs & Number of Unique Sentence IDs & Total Word Count \\
Year &  &  &  \\
\midrule
2022 & 2133 & 416312 & 10897064 \\
2023 & 223 & 32116 & 1023188 \\
2020 & 1706 & 403296 & 8869728 \\
2021 & 2270 & 517794 & 13278329 \\
2019 & 843 & 211015 & 4426701 \\
2017 & 667 & 245873 & 6671662 \\
2018 & 809 & 299043 & 6429762 \\
2014 & 400 & 189481 & 4269613 \\
2016 & 672 & 318731 & 7406029 \\
2015 & 838 & 218988 & 5348002 \\
2013 & 119 & 43958 & 1603617 \\
2012 & 44 & 21843 & 488825 \\
2011 & 21 & 5341 & 94013 \\
2010 & 6 & 4176 & 45407 \\
2009 & 2 & 2658 & 37411 \\
2005 & 1 & 1035 & 1966 \\
\bottomrule
\end{tabular}
\end{table}



In [8]:
# get a count table of doc, sentences and words by evidence_query
import json
import pandas as pd
from collections import defaultdict

def filter_and_aggregate_by_query(input_file, start_year, end_year):
    query_data = defaultdict(lambda: {'unique_document_ids': set(), 'unique_sentence_ids': set(), 'total_word_count': 0})
    
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            data = json.loads(line)
            document_id = data.get('document_id')
            sentences = data.get('sentences', [])
            meta = data.get('meta', {})
            meta_evidences = meta.get('evidences', [])
            
            for evidence_list in meta_evidences:
                for evidence in evidence_list:
                    evidence_year = evidence.get('evidence_year')
                    
                    # Filter by year
                    if start_year <= evidence_year <= end_year:
                        evidence_query = evidence.get('evidence_query', 'Unknown')
                        
                        query_data[evidence_query]['unique_document_ids'].add(document_id)
                        for sentence in sentences:
                            query_data[evidence_query]['unique_sentence_ids'].add(sentence['sentence_id'])
                            query_data[evidence_query]['total_word_count'] += len(sentence['text'].split())

    # Aggregate counts
    query_counts = {query: {'unique_document_id': len(data['unique_document_ids']),
                            'unique_sentence_id': len(data['unique_sentence_ids']),
                            'total_word_count': data['total_word_count']}
                    for query, data in query_data.items()}
    
    return query_counts

# Define the path to your combined file
combined_file = r'C:\Users\hoath\Git\LobbyMap_ML\data\processed\combined.jsonl'

# Filter and aggregate data for the years 2013 to 2023
query_counts = filter_and_aggregate_by_query(combined_file, 2013, 2023)

# Create a DataFrame from the aggregated data
df = pd.DataFrame.from_dict(query_counts, orient='index')
df.index.name = 'evidence_query'
df.columns = ['unique_document_id', 'unique_sentence_id', 'total_word_count']

# Sort the DataFrame by evidence_query for better readability
df = df.sort_index()

# Generate LaTeX source code for the table
latex_code = df.to_latex(index=True, caption="Counts of Documents, Sentences, and Words by Evidence Query (2013-2023)")

# Define the path to save the LaTeX source code
latex_file_path = r'C:\Users\hoath\Git\LobbyMap_ML\data\processed\evidence_query_counts_2013_2023.tex'

# Save the LaTeX source code to a file
with open(latex_file_path, 'w') as latex_file:
    latex_file.write(latex_code)

# Print the LaTeX code to the console
print(latex_code)


\begin{table}
\caption{Counts of Documents, Sentences, and Words by Evidence Query (2013-2023)}
\begin{tabular}{lrrr}
\toprule
 & unique_document_id & unique_sentence_id & total_word_count \\
evidence_query &  &  &  \\
\midrule
Alignment with IPCC on Climate Action & 1606 & 8068 & 8424621 \\
Carbon Tax & 604 & 7031 & 2369986 \\
Communication of Climate Science & 623 & 36217 & 4828761 \\
Disclosure on Relationships & 321 & 9129 & 2892955 \\
Emissions Trading & 863 & 15020 & 4433509 \\
Energy Transition & Zero Carbon Technologies & 5256 & 7863 & 18579214 \\
Energy and Resource Efficiency & 688 & 5060 & 2952579 \\
GHG Emission Regulation & 1348 & 13742 & 6697640 \\
Land Use & 107 & 5767 & 392355 \\
Renewable Energy & 1072 & 16322 & 4475570 \\
Support of UN Climate Process & 896 & 16322 & 5625112 \\
Supporting the Need for Regulations & 1159 & 26938 & 6056632 \\
Transparency on Legislation & 280 & 8570 & 2492885 \\
\bottomrule
\end{tabular}
\end{table}



In [4]:
# get count table by languague and evidence_query
import json
import pandas as pd
from langdetect import detect
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed

def detect_language_in_sentences(sentences, sentence_limit=50):
    language_counts = defaultdict(int)
    for sentence in sentences[:sentence_limit]:  # Limit the number of sentences processed
        text = sentence.get('text', '')
        if text:
            try:
                language = detect(text)
                language_counts[language] += 1
            except:
                pass  # If language detection fails, ignore this sentence
    return language_counts

def process_line(line, sentence_limit=50):
    language_data = defaultdict(set)
    data = json.loads(line)
    document_id = data.get('document_id')
    sentences = data.get('sentences', [])
    meta = data.get('meta', {})
    meta_evidences = meta.get('evidences', [])

    # Filter by evidence_year (only 2013-2023)
    for evidence_list in meta_evidences:
        for evidence in evidence_list:
            evidence_year = evidence.get('evidence_year')
            if 2013 <= evidence_year <= 2023:
                evidence_query = evidence.get('evidence_query', 'Unknown')
                sentence_languages = detect_language_in_sentences(sentences, sentence_limit)
                for language in sentence_languages:
                    language_data[(language, evidence_query)].add(document_id)
    
    return language_data

def process_jsonl_and_detect_language(input_file, sentence_limit=50):
    combined_language_data = defaultdict(lambda: defaultdict(set))
    
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_line, line, sentence_limit) for line in lines]

        for future in as_completed(futures):
            language_data = future.result()
            for (language, evidence_query), document_ids in language_data.items():
                combined_language_data[language][evidence_query].update(document_ids)

    # Convert the results to counts
    language_counts = {
        language: {query: len(document_ids) for query, document_ids in queries.items()}
        for language, queries in combined_language_data.items()
    }
    
    return language_counts

# Define the path to your combined file
combined_file = file_path

# Process the JSONL file and detect language
language_counts = process_jsonl_and_detect_language(combined_file)

# Convert to a DataFrame for better handling and LaTeX export
df = pd.DataFrame(language_counts).fillna(0).astype(int).T
df.index.name = 'Language'
df.columns.name = 'Evidence Query'

# Generate LaTeX source code
latex_code = df.to_latex(index=True, caption="Number of Unique Document IDs by Language and Evidence Query")

# Define the path to save the LaTeX source code
latex_file_path = r'C:\Users\hoath\Git\LobbyMap_ML\data\processed\language_counts_by_query.tex'

# Save the LaTeX source code to a file
with open(latex_file_path, 'w') as latex_file:
    latex_file.write(latex_code)

# Print the LaTeX code to the console
print(latex_code)


\begin{table}
\caption{Number of Unique Document IDs by Language and Evidence Query}
\begin{tabular}{lrrrrrrrrrrrrr}
\toprule
Evidence Query & Energy Transition & Zero Carbon Technologies & GHG Emission Regulation & Communication of Climate Science & Transparency on Legislation & Alignment with IPCC on Climate Action & Supporting the Need for Regulations & Renewable Energy & Support of UN Climate Process & Disclosure on Relationships & Carbon Tax & Energy and Resource Efficiency & Emissions Trading & Land Use \\
Language &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
af & 1382 & 284 & 126 & 53 & 372 & 232 & 197 & 177 & 61 & 109 & 132 & 149 & 25 \\
ca & 1370 & 291 & 159 & 61 & 403 & 290 & 268 & 209 & 72 & 123 & 137 & 184 & 21 \\
cs & 87 & 14 & 12 & 4 & 30 & 20 & 11 & 13 & 2 & 5 & 8 & 11 & 5 \\
cy & 897 & 255 & 71 & 115 & 221 & 171 & 195 & 102 & 146 & 138 & 160 & 178 & 26 \\
da & 959 & 188 & 105 & 31 & 280 & 180 & 129 & 127 & 33 & 90 & 102 & 102 & 18 \\
de & 2880 & 757 & 334 & 109 & 